# 队列
- 加锁可以保证多个进程修改同一块数据时，同一时间只能有一个任务可以进行修改，即串行的修改，没错，速度是慢了，但牺牲了速度却保证了数据安全。</br>
虽然可以用文件共享数据实现进程间通信，但问题是：</br>
1. 效率低（共享数据基于文件，而文件是硬盘上的数据）
2. 需要自己加锁处理

- 因此我们最好找寻一种解决方案能够兼顾：1、效率高（多个进程共享一块内存的数据）2、帮我们处理好锁问题。这就是mutiprocessing模块为我们提供的基于消息的IPC通信机制：队列和管道。
1. 队列和管道都是将数据存放于内存中
2. 队列又是基于（管道+锁）实现的，可以让我们从复杂的锁问题中解脱出来，

- 我们应该尽量避免使用共享数据，尽可能使用消息传递和队列，避免处理复杂的同步和锁问题，而且在进程数目增多时，往往可以获得更好的可获展性。

In [1]:
# 进程彼此之间互相隔离，要实现进程间通信（IPC），multiprocessing模块支持两种形式：队列和管道，这两种方式都是使用消息传递的

from multiprocessing import Queue

# Queue([maxsize]):创建共享的进程队列，Queue是多进程安全的队列，可以使用Queue实现多进程之间的数据传递。 
q=Queue(3)  # maxsize是队列中允许最大项数，省略则无大小限制。

q.put(3)
q.put(3)
q.put(3)
print(q.full()) #满了

print(q.get())
print(q.get())
print(q.get())
print(q.empty()) #空了


# q.put方法用以插入数据到队列中，put方法还有两个可选参数：blocked和timeout。如果blocked为True（默认值），并且timeout为正值，该方法会阻塞timeout指定的时间，直到该队列有剩余的空间。如果超时，会抛出Queue.Full异常。如果blocked为False，但该Queue已满，会立即抛出Queue.Full异常。
# q.get方法可以从队列读取并且删除一个元素。同样，get方法有两个可选参数：blocked和timeout。如果blocked为True（默认值），并且timeout为正值，那么在等待时间内没有取到任何元素，会抛出Queue.Empty异常。如果blocked为False，有两种情况存在，如果Queue有一个值可用，则立即返回该值，否则，如果队列为空，则立即抛出Queue.Empty异常.

# q.get_nowait():同q.get(False)
# q.put_nowait():同q.put(False)

# q.empty():调用此方法时q为空则返回True，该结果不可靠，比如在返回True的过程中，如果队列中又加入了项目。
# q.full()：调用此方法时q已满则返回True，该结果不可靠，比如在返回True的过程中，如果队列中的项目被取走。
# q.qsize():返回队列中目前项目的正确数量，结果也不可靠，理由同q.empty()和q.full()一样


True
3
3
3
True


# 生产者消费者模型
- 普通版本

In [2]:
import time

def producer():
    for i in range(1,4):
        res=f"包子{i}"
        time.sleep(2)
        print(f"生产者生产了{res}")
        consumer(res)

def consumer(res):
    time.sleep(1)
    print(f"消费者吃了{res}")

if __name__ == "__main__":
    producer()

生产者生产了包子1
消费者吃了包子1
生产者生产了包子2
消费者吃了包子2
生产者生产了包子3
消费者吃了包子3


- 队列版本</br>
1. 生产者消费者模式是通过一个容器来解决生产者和消费者的强耦合问题。
2. 生产者和消费者彼此之间不直接通讯，而通过阻塞队列来进行通讯，所以生产者生产完数据之后不用等待消费者处理，</br>直接扔给阻塞队列，消费者不找生产者要数据，而是直接从阻塞队列里取，</br>阻塞队列就相当于一个缓冲区，平衡了生产者和消费者的处理能力。

In [3]:
%%writefile test_producer_consumer.py
from multiprocessing import Queue,Process
import time

def producer(q,name,product):
    """生产者

    Parameters
    ----------
    q : _type_ queue
        _description_ 队列容器
    name : _type_
        _description_ 生产者名字
    product : _type_
        _description_ 生产的产品名字
    """
    for i in range(1,10):
        res=f"{product}{i}"
        time.sleep(0.5)
        print(f"{name} 生产了 {res}")
        q.put(res)
    #*q.put(None) 单个生产者的话，可以在这里加，多个则不行
    
def consumer(q,name):
    """消费者

    Parameters
    ----------
    q : _type_
        _description_ 队列容器
    name : _type_
        _description_ 消费者名字
    """
    while True:
        time.sleep(1.5)
        res=q.get()
        if res is None: break
        print(f"{name} 消费了 {res}")
    
if __name__=="__main__":
    #* 容器
    q=Queue()
    p1=Process(target=producer,args=(q,"生产者1","包子"))
    c1=Process(target=consumer,args=(q,"消费者1"))
    p1.start()
    c1.start()

    p1.join() #* 在主线程里发信号，需要等待生产者结束
    q.put(None) #* 加入生产者生产完毕的信号，不加的话消费者会死循环
    print("主线程")

Overwriting test_producer_consumer.py


- 多个生产者、消费者

In [4]:
%%writefile test_producer_consumer.py
from multiprocessing import Queue,Process
import time

def producer(q,name,product):
    """生产者

    Parameters
    ----------
    q : _type_ queue
        _description_ 队列容器
    name : _type_
        _description_ 生产者名字
    product : _type_
        _description_ 生产的产品名字
    """
    for i in range(1,6):
        res=f"{product}{i}"
        time.sleep(0.5)
        print(f"{name} 生产了 {res}")
        q.put(res)
        
def consumer(q,name):
    """生产者

    Parameters
    ----------
    q : _type_
        _description_ 队列容器
    name : _type_
        _description_ 消费者名字
    """
    while True:
        time.sleep(1.5)
        res=q.get()
        if res is None: break
        print(f"{name} 消费了 {res}")
    
if __name__=="__main__":
    #* 容器
    q=Queue()
    p1=Process(target=producer,args=(q,"生产者1","包子"))
    p2=Process(target=producer,args=(q,"生产者2","饺子"))
    p3=Process(target=producer,args=(q,"生产者3","粽子"))
    c1=Process(target=consumer,args=(q,"雅木茶"))
    c2=Process(target=consumer,args=(q,"悟空"))
    p1.start()
    p2.start()
    p3.start()
    c1.start()
    c2.start()

    p1.join()
    p2.join()
    p3.join()
    q.put(None) #* 有几个消费者就要发几个None,有点不优雅
    q.put(None)
    print("主线程")


Overwriting test_producer_consumer.py
